# 06/15/20 - This notebook prepares samples for modeling by determining z_lens and z_source (and D_lens and D_source) from z1 and z2, correcting M* for z_lens where Z = z_source $$M_{*correct} = M_* (D_{source}^2/D_{lens}^2)$$

In [94]:
# libraries
import numpy as np
import pandas as pd
from astropy.io import fits
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline

from astropy import constants as const
from astropy.io import fits
import astropy.units as u
from astropy.cosmology import FlatLambdaCDM, z_at_value

c = const.c.to('km/s').value #c in km/s
h0 = 73.8 #h0 in km/(Mpc*s) Riess2011
cosmo = FlatLambdaCDM(h0, Om0=0.262)

In [95]:
# read in catalogs
#links = pd.read_csv('/home/shawn/Desktop/links_autoz/files/csv/links_latest.csv')
#links_knabel = pd.read_csv('/home/shawn/Desktop/links_autoz/files/csv/links_knabel_latest.csv')
links_select= pd.read_csv('/home/shawn/Desktop/links_autoz/files/csv/links_select_latest.csv')
links_knabel_select = pd.read_csv('/home/shawn/Desktop/links_autoz/files/csv/links_knabel_select_latest.csv')
#li = pd.read_csv('/home/shawn/Desktop/links_autoz/files/csv/li_latest.csv')
li_select = pd.read_csv('/home/shawn/Desktop/links_autoz/files/csv/li_select_latest.csv')

## LinKS sample

In [96]:
links_autoz_sample = links_select
links_autoz_sample = links_autoz_sample.drop(columns = {'Unnamed: 0', 
                                                        'Unnamed: 0.1', 
                                                        'Unnamed: 0.1.1',
                                                       'Unnamed: 0.1.1.1'})
links_autoz_sample.columns

Index(['GAMA_ID', 'ID', 'RA', 'DEC', 'SCORE', 'z1', 'sigma1', 'template1',
       'z2', 'sigma2', 'template2', 'sigma3', 'sigma4', 'R', 'prob', 'error',
       'type'],
      dtype='object')

In [97]:
# need M*
# open magphys catalog
hdul = fits.open('/home/shawn/Desktop/gravitational_lensing_research/FITS_Files/MagPhys.fits')  # open a FITS file
hdul.verify('fix')
MagPhys = hdul[1].data  # assume the first extension is a table
#print(hdul[1].columns)
gama_id = MagPhys.CATAID
z = MagPhys.Z
stellar_mass = MagPhys.mass_stellar_best_fit

magphys_list = list(zip(gama_id, z, stellar_mass))

magphys = pd.DataFrame(magphys_list, columns = {'GAMA_ID','Z','STELLAR_MASS'})
#magphys = pd.DataFrame({'GAMA_ID' : gama_id,
 #                      'Z' : z,
  #                     'STELLAR_MASS' : stellar_mass})
#agphys = magphys.astype({'GAMA_ID' : 'int64'})
#magphys = magphys.rename(columns = {'Z':'GAMA_ID','GAMA_ID':'Z'})#,'1': 'Z','2': 'STELLAR_MASS'})
magphys

,GAMA_ID,Z,STELLAR_MASS
0,6802,0.0505,1.117000e+09
1,6806,0.3312,8.560000e+10
2,6808,0.2294,5.503000e+10
3,6810,0.3263,1.204000e+11
4,6813,0.4846,1.025000e+11
...,...,...,...
120109,4342359,0.6461,9.486000e+10
120110,4345057,0.7930,4.293000e+11
120111,4345978,0.6111,1.251000e+11
120112,4347483,0.5960,1.335000e+11


In [98]:
links_autoz_sample = pd.merge(links_autoz_sample, magphys, on = 'GAMA_ID')
links_autoz_sample.columns

Index(['GAMA_ID', 'ID', 'RA', 'DEC', 'SCORE', 'z1', 'sigma1', 'template1',
       'z2', 'sigma2', 'template2', 'sigma3', 'sigma4', 'R', 'prob', 'error',
       'type', 'Z', 'STELLAR_MASS'],
      dtype='object')

In [99]:
links_autoz_sample = links_autoz_sample[['GAMA_ID', 'ID', 'RA', 'DEC', 'SCORE', 
                              'type', 'Z', 'STELLAR_MASS',
                              'z1', 'sigma1', 'template1',
       'z2', 'sigma2', 'template2', 'sigma3', 'sigma4', 'R', 'prob', 'error']]
links_autoz_sample.columns

Index(['GAMA_ID', 'ID', 'RA', 'DEC', 'SCORE', 'type', 'Z', 'STELLAR_MASS',
       'z1', 'sigma1', 'template1', 'z2', 'sigma2', 'template2', 'sigma3',
       'sigma4', 'R', 'prob', 'error'],
      dtype='object')

In [25]:
links_autoz_sample[abs(links_autoz_sample.Z - links_autoz_sample.z1) > 0.05 ]

,GAMA_ID,ID,RA,DEC,SCORE,type,Z,STELLAR_MASS,z1,sigma1,template1,z2,sigma2,template2,sigma3,sigma4,R,prob,error
14,544226.0,3212.0,214.42336,-0.89247,18.0,PG + ELG,0.2270,7.340000e+10,0.649852,6.409822,47.0,0.227039,6.293928,40.0,3.835091,3.835091,1.641142,0.366429,39.200554
25,362243.0,171.0,214.01758,2.62049,4.0,PG + ELG,0.3701,1.357000e+11,0.784531,4.188629,43.0,0.370643,3.934723,40.0,3.273205,3.273205,1.202101,0.106919,75.084015


### Need zlens and zsource to match with the corresponding z1 and z2.

In [186]:
cosmo.angular_diameter_distance(0.01)

<Quantity 40.14083652 Mpc>

In [129]:
z1 = links_autoz_sample.z1
z2 = links_autoz_sample.z2
Z = links_autoz_sample.Z
stellar_mass = links_autoz_sample.STELLAR_MASS
sigma1 = links_autoz_sample.sigma1
sigma2 = links_autoz_sample.sigma2

#links_autoz_sample['zlens'] = 0
#links_autoz_sample['zsource'] = 0

zlens = []
zsource = []
sigma_lens = []
sigma_source = []
dlens = []
dsource = []
corrected_stellar_mass = []

for i in range(len(links_autoz_sample.Z)):
    if (z1[i] < z2[i]) & (abs(z1[i] - Z[i] < 0.005)): # if z1 is closer and matches to Z
        #links_autoz_sample['zlens'][i] = z1[i]
        #links_autoz_sample['zsource'][i] = z2[i]
        zl = z1[i]
        zs = z2[i]
        sigma_l = sigma1[i]
        sigma_s = sigma2[i]
        dl = cosmo.angular_diameter_distance(zl).value
        ds = cosmo.angular_diameter_distance(zs).value
        m = stellar_mass[i]
        # append lists
        zlens.append(zl)
        zsource.append(zs)
        sigma_lens.append(sigma_l)
        sigma_source.append(sigma_s)
        dlens.append(dl)
        dsource.append(ds)
        corrected_stellar_mass.append(m)
    elif (z2[i] < z1[i]) & (abs(z2[i] - Z[i]) < 0.005): # if z2 is closer and matches to Z
        #links_autoz_sample['zlens'][i] = z2[i]
        #links_autoz_sample['zsource'][i] = z1[i]
        zl = z2[i]
        zs = z1[i]
        sigma_l = sigma2[i]
        sigma_s = sigma1[i]
        dl = cosmo.angular_diameter_distance(zl).value
        ds = cosmo.angular_diameter_distance(zs).value
        m = stellar_mass[i]
        # append lists
        zlens.append(zl)
        zsource.append(zs)
        sigma_lens.append(sigma_l)
        sigma_source.append(sigma_s)
        dlens.append(dl)
        dsource.append(ds)
        corrected_stellar_mass.append(m)        
    elif (z1[i] < z2[i]) & (abs(z2[i] - Z[i]) < 0.005): # if z1 is closer, but z2 matches Z
        #print(links_autoz_sample.GAMA_ID[i], 'is wonky.')
        zl = z1[i]
        zs = z2[i]
        sigma_l = sigma1[i]
        sigma_s = sigma2[i]
        dl = cosmo.angular_diameter_distance(zl).value
        ds = cosmo.angular_diameter_distance(zs).value
        m = stellar_mass[i] * ((dl**2)/(ds**2))
        # append lists
        zlens.append(zl)
        zsource.append(zs)
        sigma_lens.append(sigma_l)
        sigma_source.append(sigma_s)
        dlens.append(dl)
        dsource.append(ds)
        corrected_stellar_mass.append(m)
    elif (z2[i] < z1[i]) & (abs(z1[i] - Z[i]) < 0.005): # if z2 is closer, but z1 matches Z
        #print(links_autoz_sample.GAMA_ID[i], 'is wonky.')
        zl = z2[i]
        zs = z1[i]
        sigma_l = sigma2[i]
        sigma_s = sigma1[i]
        dl = cosmo.angular_diameter_distance(zl).value
        ds = cosmo.angular_diameter_distance(zs).value
        m = stellar_mass[i] * ((dl**2)/(ds**2))
        # append lists
        zlens.append(zl)
        zsource.append(zs)
        sigma_lens.append(sigma_l)
        sigma_source.append(sigma_s)
        dlens.append(dl)
        dsource.append(ds)
        corrected_stellar_mass.append(m)
    else:
        print(links_autoz_sample.GAMA_ID[i], 'is wonky.')
        
        
#print(zlens, zsource, dlens, dsource, corrected_stellar_mass)

In [130]:
np.log10(corrected_stellar_mass)

array([11.21325207, 11.16016831, 11.08134732, 10.95002391, 11.51441491,
       11.12936759, 11.07261748, 11.13481438, 10.83505611, 11.09025804,
       11.34498141, 11.05307843, 11.05307843, 10.86569606, 10.86569606,
       11.24674471, 10.93490224, 11.43184605, 11.30102999, 11.221675  ,
       11.27737999, 11.58465697, 10.98987632, 11.30663945, 11.46496569,
       11.13257984, 11.22967295,  7.78785201, 10.15414463,  9.99854359,
       10.71681188,  1.36332067, 10.89354639, 10.58007321, 11.00689372,
       10.56097735,  7.45367263,  9.15857933, 10.07050901, 11.32817567,
       10.93404374, 11.55882852, 11.52296561, 11.10105934, 11.48358732,
       11.48358732, 10.9607561 , 10.9240357 , 11.46486383, 11.32159842,
       10.99852327, 11.16849748, 11.27300128, 11.24452449, 10.91871166,
       11.2380461 ])

In [131]:
links_autoz_sample['zlens'] = zlens
links_autoz_sample['zsource'] = zsource
links_autoz_sample['sigma_lens'] = sigma_lens
links_autoz_sample['sigma_source'] = sigma_source
links_autoz_sample['dlens'] = dlens
links_autoz_sample['dsource'] = dsource
links_autoz_sample['corrected_stellar_mass'] = corrected_stellar_mass

links_autoz_sample.columns

Index(['GAMA_ID', 'ID', 'RA', 'DEC', 'SCORE', 'type', 'Z', 'STELLAR_MASS',
       'z1', 'sigma1', 'template1', 'z2', 'sigma2', 'template2', 'sigma3',
       'sigma4', 'R', 'prob', 'error', 'zlens', 'zsource', 'dlens', 'dsource',
       'corrected_stellar_mass', 'sigma_lens', 'sigma_source'],
      dtype='object')

### Now reorder columns

In [132]:
links_autoz_sample = links_autoz_sample[['GAMA_ID', 'ID', 'RA', 'DEC', 'SCORE', 
                              'type', 'corrected_stellar_mass', 'zlens', 'sigma_lens',
                                         'zsource', 'sigma_source', 'R',
                                         'dlens', 'dsource', 
                                         'Z', 'STELLAR_MASS', 
                              'z1', 'sigma1', 'template1',
       'z2', 'sigma2', 'template2', 'sigma3', 'sigma4', 'prob', 'error']]
links_autoz_sample

,GAMA_ID,ID,RA,DEC,SCORE,type,corrected_stellar_mass,zlens,sigma_lens,zsource,...,z1,sigma1,template1,z2,sigma2,template2,sigma3,sigma4,prob,error
0,600305.0,245.0,134.75916,0.23814,12.0,PG + ELG,1.634000e+11,0.243359,10.951900,0.634666,...,0.243359,10.951900,40.0,0.634666,3.553775,45.0,2.635769,2.635769,1.000000,40.554672
1,419067.0,1179.0,138.61974,2.63477,8.0,PG + ELG,1.446000e+11,0.187679,11.541553,0.763765,...,0.187679,11.541553,40.0,0.763765,4.008492,43.0,2.866317,2.866317,1.000000,44.729992
2,387244.0,1819.0,135.56917,2.36495,4.0,PG + ELG,1.206000e+11,0.217723,12.368897,0.712327,...,0.217723,12.368897,40.0,0.712327,3.347791,47.0,2.785462,2.785462,1.000000,38.658886
3,3611088.0,1743.0,135.38005,-1.29831,4.0,PG + ELG,8.913000e+10,0.231903,9.052866,0.771556,...,0.231903,9.052866,40.0,0.771556,4.793196,43.0,3.291919,3.291919,0.999928,43.360439
4,373093.0,2897.0,139.30637,1.19776,4.0,PG + ELG,3.269000e+11,0.383991,8.714600,0.837412,...,0.383991,8.714600,40.0,0.837412,4.373900,43.0,3.555707,3.555707,0.999842,54.197964
5,585328.0,3234.0,182.36838,-0.00889,18.0,PG + ELG,1.347000e+11,0.204663,11.430782,0.788741,...,0.204663,11.430782,40.0,0.788741,3.371675,43.0,2.756129,2.756129,1.000000,39.344978
6,561058.0,3349.0,182.56030,-0.49540,8.0,PG + ELG,1.182000e+11,0.320287,9.753261,0.855552,...,0.320287,9.753261,40.0,0.855552,3.635149,43.0,3.017413,3.017413,1.000000,48.403797
7,84425.0,2562.0,177.77478,0.42791,8.0,PG + ELG,1.364000e+11,0.285867,12.257433,0.673673,...,0.285867,12.257433,40.0,0.673673,3.122599,43.0,2.550145,2.550145,1.000000,38.429665
8,165293.0,2144.0,180.60150,-2.74642,8.0,PG + ELG,6.840000e+10,0.259462,9.833871,0.870761,...,0.259462,9.833871,40.0,0.870761,3.698742,43.0,2.978528,2.978528,1.000000,45.051468
9,559216.0,2507.0,176.11624,-0.61902,4.0,PG + ELG,1.231000e+11,0.250201,9.004106,0.713636,...,0.250201,9.004106,40.0,0.713636,3.653380,45.0,2.756443,2.756443,0.999999,50.808048


In [136]:
len(links_autoz_sample[links_autoz_sample.STELLAR_MASS != links_autoz_sample.corrected_stellar_mass])

18

### 18 of the 50 had to be corrected because Z from magphys matched the z_source.

In [137]:
links_autoz_sample.columns

Index(['GAMA_ID', 'ID', 'RA', 'DEC', 'SCORE', 'type', 'corrected_stellar_mass',
       'zlens', 'sigma_lens', 'zsource', 'sigma_source', 'R', 'dlens',
       'dsource', 'Z', 'STELLAR_MASS', 'z1', 'sigma1', 'template1', 'z2',
       'sigma2', 'template2', 'sigma3', 'sigma4', 'prob', 'error'],
      dtype='object')

In [138]:
#links_autoz_sample.to_csv('/home/shawn/Desktop/links_autoz/files/csv/links_autoz_sample_061520.csv')
#links_autoz_sample.to_csv('/home/shawn/Desktop/links_autoz/files/csv/links_autoz_sample_latest.csv')

### Now for the Knabel2020 LinKS candidates.

In [148]:
links_knabel_select

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,GAMA_ID,ID,RA,DEC,SCORE,STELLAR_MASS,...,template1,z2,sigma2,template2,sigma3,sigma4,R,prob,error,type
0,3,34,34,31,585328.0,3234.0,182.36838,-0.00889,18.0,1.347000e+11,...,40.0,0.788741,3.371675,43.0,2.756129,2.756129,1.223337,1.000000,39.344978,PG + ELG
1,4,48,48,45,544226.0,3212.0,214.42336,-0.89247,18.0,7.340000e+10,...,40.0,0.649824,7.239896,46.0,3.411552,3.411552,2.122170,0.996206,45.355965,PG + ELG
2,6,50,50,46,250289.0,2730.0,214.36708,1.99311,28.0,1.894000e+11,...,40.0,0.719616,4.515500,47.0,3.275637,3.275637,1.378510,0.999952,52.423916,PG + ELG
3,0,2,2,2,323152.0,2967.0,130.54643,1.64325,30.0,8.277000e+11,...,47.0,0.353286,7.519264,40.0,4.763242,4.763242,1.578602,0.989458,29.413359,PG + ELG
4,5,49,49,45,544226.0,3212.0,214.42336,-0.89247,18.0,7.340000e+10,...,47.0,0.227039,6.293928,40.0,3.835091,3.835091,1.641142,0.366429,39.200554,PG + ELG
5,2,6,6,5,3575500.0,1906.0,129.23265,-1.57512,26.0,3.209000e+11,...,42.0,0.232537,4.330916,43.0,3.326265,3.326265,1.302036,0.999702,57.593048,ELG + PG
6,1,4,4,4,3629152.0,1933.0,135.88864,-0.97487,26.0,2.129000e+11,...,40.0,0.787118,3.691353,40.0,3.029998,3.029998,1.218269,0.792829,87.774857,ELG + ELG


In [149]:
links_knabel_select.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1',
       'GAMA_ID', 'ID', 'RA', 'DEC', 'SCORE', 'STELLAR_MASS', 'Z', 'theta_e',
       'z1', 'sigma1', 'template1', 'z2', 'sigma2', 'template2', 'sigma3',
       'sigma4', 'R', 'prob', 'error', 'type'],
      dtype='object')

In [150]:
links_knabel_autoz_sample = links_knabel_select
links_knabel_autoz_sample = links_knabel_autoz_sample.drop(columns = {'Unnamed: 0', 
                                                        'Unnamed: 0.1', 
                                                        'Unnamed: 0.1.1',
                                                       'Unnamed: 0.1.1.1'})
links_knabel_autoz_sample.columns

Index(['GAMA_ID', 'ID', 'RA', 'DEC', 'SCORE', 'STELLAR_MASS', 'Z', 'theta_e',
       'z1', 'sigma1', 'template1', 'z2', 'sigma2', 'template2', 'sigma3',
       'sigma4', 'R', 'prob', 'error', 'type'],
      dtype='object')

In [151]:
links_knabel_autoz_sample

,GAMA_ID,ID,RA,DEC,SCORE,STELLAR_MASS,Z,theta_e,z1,sigma1,template1,z2,sigma2,template2,sigma3,sigma4,R,prob,error,type
0,585328.0,3234.0,182.36838,-0.00889,18.0,1.347000e+11,0.2047,1.299258,0.204663,11.430782,40.0,0.788741,3.371675,43.0,2.756129,2.756129,1.223337,1.000000,39.344978,PG + ELG
1,544226.0,3212.0,214.42336,-0.89247,18.0,7.340000e+10,0.2270,0.910764,0.227044,9.393048,40.0,0.649824,7.239896,46.0,3.411552,3.411552,2.122170,0.996206,45.355965,PG + ELG
2,250289.0,2730.0,214.36708,1.99311,28.0,1.894000e+11,0.4009,1.100886,0.401214,8.795989,40.0,0.719616,4.515500,47.0,3.275637,3.275637,1.378510,0.999952,52.423916,PG + ELG
3,323152.0,2967.0,130.54643,1.64325,30.0,8.277000e+11,0.7216,1.715374,0.721590,11.317520,47.0,0.353286,7.519264,40.0,4.763242,4.763242,1.578602,0.989458,29.413359,PG + ELG
4,544226.0,3212.0,214.42336,-0.89247,18.0,7.340000e+10,0.2270,0.910764,0.649852,6.409822,47.0,0.227039,6.293928,40.0,3.835091,3.835091,1.641142,0.366429,39.200554,PG + ELG
5,3575500.0,1906.0,129.23265,-1.57512,26.0,3.209000e+11,0.3652,1.501378,0.365195,8.068478,42.0,0.232537,4.330916,43.0,3.326265,3.326265,1.302036,0.999702,57.593048,ELG + PG
6,3629152.0,1933.0,135.88864,-0.97487,26.0,2.129000e+11,0.4059,1.159976,0.406623,4.328009,40.0,0.787118,3.691353,40.0,3.029998,3.029998,1.218269,0.792829,87.774857,ELG + ELG


In [157]:
z1 = links_knabel_autoz_sample.z1
z2 = links_knabel_autoz_sample.z2
Z = links_knabel_autoz_sample.Z
stellar_mass = links_knabel_autoz_sample.STELLAR_MASS
sigma1 = links_knabel_autoz_sample.sigma1
sigma2 = links_knabel_autoz_sample.sigma2

zlens = []
zsource = []
sigma_lens = []
sigma_source = []
dlens = []
dsource = []
corrected_stellar_mass = []

for i in range(len(links_knabel_autoz_sample.Z)):
    if (z1[i] < z2[i]) & (abs(z1[i] - Z[i] < 0.005)): # if z1 is closer and matches to Z
        #links_autoz_sample['zlens'][i] = z1[i]
        #links_autoz_sample['zsource'][i] = z2[i]
        zl = z1[i]
        zs = z2[i]
        sigma_l = sigma1[i]
        sigma_s = sigma2[i]
        dl = cosmo.angular_diameter_distance(zl).value
        ds = cosmo.angular_diameter_distance(zs).value
        m = stellar_mass[i]
        # append lists
        zlens.append(zl)
        zsource.append(zs)
        sigma_lens.append(sigma_l)
        sigma_source.append(sigma_s)
        dlens.append(dl)
        dsource.append(ds)
        corrected_stellar_mass.append(m)
    elif (z2[i] < z1[i]) & (abs(z2[i] - Z[i]) < 0.005): # if z2 is closer and matches to Z
        #links_autoz_sample['zlens'][i] = z2[i]
        #links_autoz_sample['zsource'][i] = z1[i]
        zl = z2[i]
        zs = z1[i]
        sigma_l = sigma2[i]
        sigma_s = sigma1[i]
        dl = cosmo.angular_diameter_distance(zl).value
        ds = cosmo.angular_diameter_distance(zs).value
        m = stellar_mass[i]
        # append lists
        zlens.append(zl)
        zsource.append(zs)
        sigma_lens.append(sigma_l)
        sigma_source.append(sigma_s)
        dlens.append(dl)
        dsource.append(ds)
        corrected_stellar_mass.append(m)        
    elif (z1[i] < z2[i]) & (abs(z2[i] - Z[i]) < 0.005): # if z1 is closer, but z2 matches Z
        #print(links_autoz_sample.GAMA_ID[i], 'is wonky.')
        zl = z1[i]
        zs = z2[i]
        sigma_l = sigma1[i]
        sigma_s = sigma2[i]
        dl = cosmo.angular_diameter_distance(zl).value
        ds = cosmo.angular_diameter_distance(zs).value
        m = stellar_mass[i] * ((dl**2)/(ds**2))
        # append lists
        zlens.append(zl)
        zsource.append(zs)
        sigma_lens.append(sigma_l)
        sigma_source.append(sigma_s)
        dlens.append(dl)
        dsource.append(ds)
        corrected_stellar_mass.append(m)
    elif (z2[i] < z1[i]) & (abs(z1[i] - Z[i]) < 0.005): # if z2 is closer, but z1 matches Z
        #print(links_autoz_sample.GAMA_ID[i], 'is wonky.')
        zl = z2[i]
        zs = z1[i]
        sigma_l = sigma2[i]
        sigma_s = sigma1[i]
        dl = cosmo.angular_diameter_distance(zl).value
        ds = cosmo.angular_diameter_distance(zs).value
        m = stellar_mass[i] * ((dl**2)/(ds**2))
        # append lists
        zlens.append(zl)
        zsource.append(zs)
        sigma_lens.append(sigma_l)
        sigma_source.append(sigma_s)
        dlens.append(dl)
        dsource.append(ds)
        corrected_stellar_mass.append(m)
    else:
        print(links_knabel_autoz_sample.GAMA_ID[i], 'is wonky.')
        
        
#print(zlens, zsource, dlens, dsource, corrected_stellar_mass)

In [158]:
zlens

[0.20466271042823792,
 0.22704437375068665,
 0.4012139439582825,
 0.35328593850135803,
 0.2270387411117553,
 0.23253659904003146,
 0.4066231548786163]

In [159]:
links_knabel_autoz_sample['zlens'] = zlens
links_knabel_autoz_sample['zsource'] = zsource
links_knabel_autoz_sample['sigma_lens'] = sigma_lens
links_knabel_autoz_sample['sigma_source'] = sigma_source
links_knabel_autoz_sample['dlens'] = dlens
links_knabel_autoz_sample['dsource'] = dsource
links_knabel_autoz_sample['corrected_stellar_mass'] = corrected_stellar_mass

links_knabel_autoz_sample.columns

Index(['GAMA_ID', 'ID', 'RA', 'DEC', 'SCORE', 'STELLAR_MASS', 'Z', 'theta_e',
       'z1', 'sigma1', 'template1', 'z2', 'sigma2', 'template2', 'sigma3',
       'sigma4', 'R', 'prob', 'error', 'type', 'zlens', 'zsource',
       'sigma_lens', 'sigma_source', 'dlens', 'dsource',
       'corrected_stellar_mass'],
      dtype='object')

In [161]:
links_knabel_autoz_sample.columns

Index(['GAMA_ID', 'ID', 'RA', 'DEC', 'SCORE', 'STELLAR_MASS', 'Z', 'theta_e',
       'z1', 'sigma1', 'template1', 'z2', 'sigma2', 'template2', 'sigma3',
       'sigma4', 'R', 'prob', 'error', 'type', 'zlens', 'zsource',
       'sigma_lens', 'sigma_source', 'dlens', 'dsource',
       'corrected_stellar_mass'],
      dtype='object')

In [162]:
links_knabel_autoz_sample = links_knabel_autoz_sample[['GAMA_ID', 'ID', 'RA', 'DEC', 'SCORE', 
                              'type', 'corrected_stellar_mass', 'zlens', 'sigma_lens',
                                         'zsource', 'sigma_source', 'R',
                                         'dlens', 'dsource', 
                                         'Z', 'STELLAR_MASS', 'theta_e', 
                              'z1', 'sigma1', 'template1',
       'z2', 'sigma2', 'template2', 'sigma3', 'sigma4', 'prob', 'error']]



In [165]:
links_knabel_autoz_sample[links_knabel_autoz_sample.corrected_stellar_mass != links_knabel_autoz_sample.STELLAR_MASS]

,GAMA_ID,ID,RA,DEC,SCORE,type,corrected_stellar_mass,zlens,sigma_lens,zsource,...,z1,sigma1,template1,z2,sigma2,template2,sigma3,sigma4,prob,error
3,323152.0,2967.0,130.54643,1.64325,30.0,PG + ELG,3.842881e+11,0.353286,7.519264,0.721590,...,0.721590,11.317520,47.0,0.353286,7.519264,40.0,4.763242,4.763242,0.989458,29.413359
5,3575500.0,1906.0,129.23265,-1.57512,26.0,ELG + PG,1.696965e+11,0.232537,4.330916,0.365195,...,0.365195,8.068478,42.0,0.232537,4.330916,43.0,3.326265,3.326265,0.999702,57.593048


In [166]:
links_knabel_autoz_sample

,GAMA_ID,ID,RA,DEC,SCORE,type,corrected_stellar_mass,zlens,sigma_lens,zsource,...,z1,sigma1,template1,z2,sigma2,template2,sigma3,sigma4,prob,error
0,585328.0,3234.0,182.36838,-0.00889,18.0,PG + ELG,1.347000e+11,0.204663,11.430782,0.788741,...,0.204663,11.430782,40.0,0.788741,3.371675,43.0,2.756129,2.756129,1.000000,39.344978
1,544226.0,3212.0,214.42336,-0.89247,18.0,PG + ELG,7.340000e+10,0.227044,9.393048,0.649824,...,0.227044,9.393048,40.0,0.649824,7.239896,46.0,3.411552,3.411552,0.996206,45.355965
2,250289.0,2730.0,214.36708,1.99311,28.0,PG + ELG,1.894000e+11,0.401214,8.795989,0.719616,...,0.401214,8.795989,40.0,0.719616,4.515500,47.0,3.275637,3.275637,0.999952,52.423916
3,323152.0,2967.0,130.54643,1.64325,30.0,PG + ELG,3.842881e+11,0.353286,7.519264,0.721590,...,0.721590,11.317520,47.0,0.353286,7.519264,40.0,4.763242,4.763242,0.989458,29.413359
4,544226.0,3212.0,214.42336,-0.89247,18.0,PG + ELG,7.340000e+10,0.227039,6.293928,0.649852,...,0.649852,6.409822,47.0,0.227039,6.293928,40.0,3.835091,3.835091,0.366429,39.200554
5,3575500.0,1906.0,129.23265,-1.57512,26.0,ELG + PG,1.696965e+11,0.232537,4.330916,0.365195,...,0.365195,8.068478,42.0,0.232537,4.330916,43.0,3.326265,3.326265,0.999702,57.593048
6,3629152.0,1933.0,135.88864,-0.97487,26.0,ELG + ELG,2.129000e+11,0.406623,4.328009,0.787118,...,0.406623,4.328009,40.0,0.787118,3.691353,40.0,3.029998,3.029998,0.792829,87.774857


### 2 of 7 had to be corrected because Z matched to z_source

In [169]:
#links_knabel_autoz_sample.to_csv('/home/shawn/Desktop/links_autoz/files/csv/links_knabel_autoz_sample_061520.csv')
#links_knabel_autoz_sample.to_csv('/home/shawn/Desktop/links_autoz/files/csv/links_knabel_autoz_sample_latest.csv')

### Li candidates

In [167]:
li_autoz_sample = li_select

z1 = li_autoz_sample.z1
z2 = li_autoz_sample.z2
Z = li_autoz_sample.Z
stellar_mass = li_autoz_sample.STELLAR_MASS
sigma1 = li_autoz_sample.sigma1
sigma2 = li_autoz_sample.sigma2

zlens = []
zsource = []
sigma_lens = []
sigma_source = []
dlens = []
dsource = []
corrected_stellar_mass = []

for i in range(len(li_autoz_sample.Z)):
    if (z1[i] < z2[i]) & (abs(z1[i] - Z[i] < 0.005)): # if z1 is closer and matches to Z
        #links_autoz_sample['zlens'][i] = z1[i]
        #links_autoz_sample['zsource'][i] = z2[i]
        zl = z1[i]
        zs = z2[i]
        sigma_l = sigma1[i]
        sigma_s = sigma2[i]
        dl = cosmo.angular_diameter_distance(zl).value
        ds = cosmo.angular_diameter_distance(zs).value
        m = stellar_mass[i]
        # append lists
        zlens.append(zl)
        zsource.append(zs)
        sigma_lens.append(sigma_l)
        sigma_source.append(sigma_s)
        dlens.append(dl)
        dsource.append(ds)
        corrected_stellar_mass.append(m)
    elif (z2[i] < z1[i]) & (abs(z2[i] - Z[i]) < 0.005): # if z2 is closer and matches to Z
        #links_autoz_sample['zlens'][i] = z2[i]
        #links_autoz_sample['zsource'][i] = z1[i]
        zl = z2[i]
        zs = z1[i]
        sigma_l = sigma2[i]
        sigma_s = sigma1[i]
        dl = cosmo.angular_diameter_distance(zl).value
        ds = cosmo.angular_diameter_distance(zs).value
        m = stellar_mass[i]
        # append lists
        zlens.append(zl)
        zsource.append(zs)
        sigma_lens.append(sigma_l)
        sigma_source.append(sigma_s)
        dlens.append(dl)
        dsource.append(ds)
        corrected_stellar_mass.append(m)        
    elif (z1[i] < z2[i]) & (abs(z2[i] - Z[i]) < 0.005): # if z1 is closer, but z2 matches Z
        #print(links_autoz_sample.GAMA_ID[i], 'is wonky.')
        zl = z1[i]
        zs = z2[i]
        sigma_l = sigma1[i]
        sigma_s = sigma2[i]
        dl = cosmo.angular_diameter_distance(zl).value
        ds = cosmo.angular_diameter_distance(zs).value
        m = stellar_mass[i] * ((dl**2)/(ds**2))
        # append lists
        zlens.append(zl)
        zsource.append(zs)
        sigma_lens.append(sigma_l)
        sigma_source.append(sigma_s)
        dlens.append(dl)
        dsource.append(ds)
        corrected_stellar_mass.append(m)
    elif (z2[i] < z1[i]) & (abs(z1[i] - Z[i]) < 0.005): # if z2 is closer, but z1 matches Z
        #print(links_autoz_sample.GAMA_ID[i], 'is wonky.')
        zl = z2[i]
        zs = z1[i]
        sigma_l = sigma2[i]
        sigma_s = sigma1[i]
        dl = cosmo.angular_diameter_distance(zl).value
        ds = cosmo.angular_diameter_distance(zs).value
        m = stellar_mass[i] * ((dl**2)/(ds**2))
        # append lists
        zlens.append(zl)
        zsource.append(zs)
        sigma_lens.append(sigma_l)
        sigma_source.append(sigma_s)
        dlens.append(dl)
        dsource.append(ds)
        corrected_stellar_mass.append(m)
    else:
        print(li_autoz_sample.GAMA_ID[i], 'is wonky.')
        
        
#print(zlens, zsource, dlens, dsource, corrected_stellar_mass)

In [168]:
li_autoz_sample['zlens'] = zlens
li_autoz_sample['zsource'] = zsource
li_autoz_sample['sigma_lens'] = sigma_lens
li_autoz_sample['sigma_source'] = sigma_source
li_autoz_sample['dlens'] = dlens
li_autoz_sample['dsource'] = dsource
li_autoz_sample['corrected_stellar_mass'] = corrected_stellar_mass

li_autoz_sample.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'name', 'KIDS_ID', 'RA', 'DEC', 'mag',
       'mag_err', 'zphot', 'z_min', 'z_max', 'score', 'rms', 'probability',
       's*p/10', 'num_0', 'GAMA_ID', 'Z', 'STELLAR_MASS', 'f_dm',
       'theta_e_Ang_slide', 'D1', 'D2', 'z1', 'sigma1', 'template1', 'z2',
       'sigma2', 'template2', 'sigma3', 'sigma4', 'R', 'prob', 'error', 'type',
       'zlens', 'zsource', 'sigma_lens', 'sigma_source', 'dlens', 'dsource',
       'corrected_stellar_mass'],
      dtype='object')

In [170]:
li_autoz_sample = li_autoz_sample.drop(columns = {'Unnamed: 0', 'Unnamed: 0.1'})
li_autoz_sample.columns

Index(['name', 'KIDS_ID', 'RA', 'DEC', 'mag', 'mag_err', 'zphot', 'z_min',
       'z_max', 'score', 'rms', 'probability', 's*p/10', 'num_0', 'GAMA_ID',
       'Z', 'STELLAR_MASS', 'f_dm', 'theta_e_Ang_slide', 'D1', 'D2', 'z1',
       'sigma1', 'template1', 'z2', 'sigma2', 'template2', 'sigma3', 'sigma4',
       'R', 'prob', 'error', 'type', 'zlens', 'zsource', 'sigma_lens',
       'sigma_source', 'dlens', 'dsource', 'corrected_stellar_mass'],
      dtype='object')

In [171]:
li_autoz_sample = li_autoz_sample[['GAMA_ID', 'name', 'KIDS_ID', 'RA', 'DEC', 
                                   'score', 
        'type', 'corrected_stellar_mass', 'zlens', 'sigma_lens', 'zsource', 'sigma_source', 
       'R', 'dlens', 'dsource',  
                                   'Z', 'STELLAR_MASS', 'f_dm', 'theta_e_Ang_slide', 'D1', 'D2', 'z1',
       'sigma1', 'template1', 'z2', 'sigma2', 'template2', 
                                   'sigma3', 'sigma4', 'prob', 'error',
                                  'mag', 'mag_err', 'zphot', 'z_min',
                                    'probability', 's*p/10', 'num_0',
       'z_max', 'rms', ]]

In [172]:
li_autoz_sample

,GAMA_ID,name,KIDS_ID,RA,DEC,score,type,corrected_stellar_mass,zlens,sigma_lens,...,error,mag,mag_err,zphot,z_min,probability,s*p/10,num_0,z_max,rms
0,16104,galaxy_217.6782_0.7445.fits.png,KiDSDR4 J143042.767+004440.22,217.678198,0.744506,4.0,PG + ELG,8.532000e+10,0.286557,11.283794,...,37.654591,18.9703,0.005238,0.41,0.37,0.998,0.399,1.0,0.44,2.683
1,362314,LRG_214.50005_2.6637.fits.png,KiDSDR4 J141800.011+023949.18,214.500046,2.663662,1.2,PG + ELG,2.446000e+11,0.367197,7.792590,...,58.424477,18.9243,0.005114,0.47,0.44,0.774,0.093,3.0,0.49,1.470
2,125108,galaxy_182.57383_-2.5036.fits.png,KiDSDR4 J121017.719-023013.12,182.573831,-2.503647,4.8,ELG + PG,6.395206e+10,0.263391,4.125181,...,52.292130,19.1443,0.005102,0.47,0.43,0.989,0.475,1.0,0.49,2.857
3,261815,LRG_216.3946_2.5116.fits.png,KiDSDR4 J142534.703+023041.84,216.394596,2.511625,1.8,ELG + PG,2.933839e+09,0.035371,3.800428,...,48.743893,18.4645,0.004361,0.37,0.34,0.904,0.163,2.0,0.39,1.470
4,55728,galaxy_183.30933_-0.243.fits.png,KiDSDR4 J121314.238-001434.63,183.309326,-0.242953,5.4,PG + PG,1.125984e+11,0.262110,5.176181,...,28.582939,18.5808,0.003280,0.46,0.42,0.982,0.530,0.0,0.48,1.960
5,39881,galaxy_178.56122_-0.7891.fits.png,KiDSDR4 J115414.693-004720.92,178.561221,-0.789145,2.6,PG + PG,1.917852e+10,0.079985,6.895601,...,36.621670,19.0313,0.005803,0.44,0.41,1.000,0.260,2.0,0.59,2.577
6,569641,galaxy_219.72969_-0.5973.fits.png,KiDSDR4 J143855.125-003550.15,219.729690,-0.597266,4.6,PG + PG,1.698000e+11,0.359714,11.781016,...,31.878830,18.2356,0.003434,0.39,0.36,0.998,0.459,0.0,0.41,1.960
7,297460,galaxy_215.90358_1.4075.fits.png,KiDSDR4 J142336.859+012427.08,215.903581,1.407523,2.0,PG + PG,1.025650e+11,0.298390,5.489465,...,26.391441,18.7758,0.004714,0.63,0.59,0.992,0.198,3.0,0.65,2.757


In [173]:
len(li_autoz_sample[li_autoz_sample.corrected_stellar_mass != li_autoz_sample.STELLAR_MASS])

5

### 5 of 8 had to be corrected because Z matched to z_source.

In [174]:
li_autoz_sample.Z

0    0.2866
1    0.3672
2    0.3397
3    0.2968
4    0.2915
5    0.4499
6    0.3597
7    0.3285
Name: Z, dtype: float64

In [177]:
#li_autoz_sample.to_csv('/home/shawn/Desktop/links_autoz/files/csv/li_autoz_sample_061520.csv')
#li_autoz_sample.to_csv('/home/shawn/Desktop/links_autoz/files/csv/li_autoz_sample_latest.csv')